# Data Preparation for KOREAN Datasets
. [AIHub](http://www.aihub.or.kr/aidata/105)에서 제공되는 데이터셋에 대한 전처리 및 가공 단계를 다루는 과정입니다.
. 이 문서에서 포함하는 내용은 다음과 같습니다.
  - Data Load                    (O)
  - Transcript 전처리 및 Label 생성 (O)
  - 오디오 전처리(MFCC) 및 저장       (X)

## 1. Data Load
- Audio File : 16k / PCM / 16bit LE
- Transcript File : EUC-KR

In [1]:
import os
import re
import numpy as np
import pandas as pd

In [2]:
audio_ext = ".pcm"
trans_ext = ".txt"

data_dir = "../data/KsponSpeech_sample/"
data_dir = "../data/"

SAMPLE_RATE = 16000
ENDIAN = "int16"
ENCODING = "euc-kr"

In [3]:
def get_file_list(path, audio_ext='.pcm', trans_ext='.txt'):
    if not os.path.exists(path):
        return []
    
    file_list = []
    for _path, _dir, _files in os.walk(path):
        for f in _files:
            if f[-len(audio_ext):] == audio_ext:
                f_name = os.path.join(_path, f[:-len(audio_ext)])
                if os.path.exists(f_name + trans_ext):
                    file_list.append(f_name)
    file_list.sort()
    
    return file_list

def get_transcript(fname, trans_ext='.txt', ENCODING='euc-kr'):
    txt = ""
    try:
        with open(fname + trans_ext, 'r', encoding=ENCODING) as f:
            txt = f.read().strip()
    except Exception as e:
        print("ERROR: {}".format(e))
    return txt

def get_audiobuff(fname, audio_ext='.pcm', ENDIAN='int16'):
    buff = None
    try:
        with open(fname + audio_ext, 'rb') as f:
            buff = f.read()
        buff = np.frombuffer(buff, dtype=ENDIAN)
    except Exception as e:
        print("ERROR: {}".format(e))
    return buff

In [4]:
file_list = get_file_list(data_dir, audio_ext, trans_ext)

print(" Audio format.     : {}".format(audio_ext))
print(" Transcript format : {}".format(trans_ext))
print(" Number of files   : {}".format(len(file_list)))
print(" Head of file list")
for f in file_list[:5]:
    print("   ", f)

 Audio format.     : .pcm
 Transcript format : .txt
 Number of files   : 100
 Head of file list
    ../data/KsponSpeech_sample/KsponSpeech_000001
    ../data/KsponSpeech_sample/KsponSpeech_000002
    ../data/KsponSpeech_sample/KsponSpeech_000003
    ../data/KsponSpeech_sample/KsponSpeech_000004
    ../data/KsponSpeech_sample/KsponSpeech_000005


In [5]:
sample_txt = get_transcript(file_list[0])
sample_audio = get_audiobuff(file_list[0])

print(" Loaded trans : {}".format(sample_txt))
print(" Loaded audio length : {}".format(len(sample_audio)))

 Loaded trans : 아/ 몬 소리야, 그건 또. b/
 Loaded audio length : 50368


## 2. Analyze Transcript
<b> Noise </b><br>
- b/  : breath   -> 숨소리
- n/  : noise    -> 노이즈
- l/  : laugh    -> 웃음소리
- o/  : occlude  -> 다른 사람의 목소리가 섞여있을 때

<b> Number </b><br>
ex)
- (5대)/(오 대) 그룹이 모여, 자동차 (5대)/(다섯 대)를
- (24시간)/(이십 사 시간), (24시간)/(스물 네 시간) -(867-860-2437)/(팔 육 칠 팔 육 공 에 이 사 삼 칠)
- (14시)/(십 사 시), (14시)/(열 네 시)부터
- (1999년)/(천 구백 구십 구 년)에, (1999년)/(일천 구백 구십 구 년)에

<b> ETC </b><br>
- '+' : 중복 발성     [AMBIG:DUP]
- '*' : 불분명한 발성  [AMBIG:UNK]
- 뭐/, 음/, 아/ 등과 같이 [한글]/ 형태의 경우 간투어를 의미함  [GANTU]

### 2.1. Transcript Preprocessing

In [6]:
for file in file_list[:15]:
    sample_txt = get_transcript(file)
    print("  ", sample_txt)

   아/ 몬 소리야, 그건 또. b/
   나는 악습은 원래 없어진다+ 없어져야 된다고 생각하긴 했는데 근데 그/ 약간 필요악으로 하나 정도쯤은 있어야 되거든. 물 뜨러 가고.
   b/ n/ 그래서 지호랑 계단 n/ 올라와서 b/ 막 위에 운동하는 기구 있대요. b/ 그서 그걸로 운동 할려구요. b/ n/
   뭐/ 정신과 병원도 그 약 타서 먹어보고, 그 한동안 연락이 안 된 적이 있었단 말이야. 그때가 언제였 언제였더라?
   o/ b/ 그게 (0.1프로)/(영 점 일 프로) 가정의 아이들과 가정의 모습이야? b/
   그/ 친애하는 판사님께라는 법+ 법 관련 드라마 알고 있어?
   o/ 그래가지고 진짜 차 사야겠다 아니 뭐/ 차 안 되면 스쿠터라도 타야되겠다 막/ 그런 생각 들더라구 그래서 운전은 하는 게 좋은 거 같애 진짜 b/
   그래
   o/ 나도 몰라. 나 그/ (3G)/(쓰리 쥐)* 하나도 안 봤음. 어.
   아/ 내일 나 알바하구나.
   n/ 아/ 근데 (1시)/(한 시)에 닫는 게 쫌 아쉽긴 한데 거기 진짜 괜찮은데 b/
   맞아. 시간 안에 풀고 약간 이것부터 풀고 요런 식으로 풀어보렴. 이런 거 b/
   삼 점대야? 언니가?
   한+ 한+ 한 시간에 이 만 원? 거의 이 정도로 이 정도란 말이야. b/
   굳이 싶기도 해. 까* 오히려 사원 이런 것보다는 b/ 대신 돔* 먹+ 먹, 음식 거리, 먹거리가 우리 입맛에 좀 더 맞는 거 같고.


In [7]:
sample_txt = "o/ b/ 그게 (0.1프로)/(영 점 일 프로)와 (15,3프로)/(십오 점 삼 프로) 가정의 아이들과 가정의 모습이야? b/"
print(sample_txt)

o/ b/ 그게 (0.1프로)/(영 점 일 프로)와 (15,3프로)/(십오 점 삼 프로) 가정의 아이들과 가정의 모습이야? b/


In [8]:
args = re.findall("\(.+?\)/\(.+?\)", sample_txt)
args

['(0.1프로)/(영 점 일 프로)', '(15,3프로)/(십오 점 삼 프로)']

In [9]:
def normStep1(txt):
    # (0.1프로)/(영 점 일 프로) or (3G)/(쓰리 쥐) or (1시)/(한 시) 등과 같은
    # (실제 표기)/(발음 표기) 형태의 데이터를 발음 표기의 데이터로 치환하는 함수
    args = re.findall("\(.+?\)/\(.+?\)", txt)
    for arg in args:
        _arg = re.sub("\(.+?\)/", "", arg)
        _arg = re.sub("[\(\)]", "", _arg)
        txt = txt.replace(arg, _arg)
    return txt

def normStep2(txt):
    # . , ? 등 특수문자 제거
    txt = re.sub("[.,?]", "", txt)
    return txt

def normStep3(txt):
    # noise symbol을 [NOISE:o], [NOIST:b] 등과 같이 변환
    symbol_list = re.findall("[a-z]/", txt)
    for symbol in symbol_list:
        _symbol = symbol.replace('/', '')
        new_symbol = "[NOISE:{}]".format(_symbol)
        txt = txt.replace(symbol, new_symbol)
    return txt

def normStep4(txt):
    # +, *와 같은 불분명한 symbol에 대한 전처리
    txt = re.sub("[\+]", "[AMBIG:DUP]", txt)
    txt = re.sub("[\*]", "[AMBIG:UNK]", txt)
    return txt
    
def normStep5(txt):
    # 뭐/ 아/ 와 같은 간투어 처리
    for _txt in txt.split():
        symbol_list = re.findall("[가-힣].*?/", _txt)
        for symbol in symbol_list:
            new_symbol = symbol.replace('/', '[GANTU]')
            txt = txt.replace(symbol, new_symbol)
    return txt

def normalize(txt):
    # step1 : to pronunciation
    txt = normStep1(txt)
    # step2 : erase special symbol
    txt = normStep2(txt)
    # step3 : noise tagging
    txt = normStep3(txt)
    # step4 : ambiguous symbol tagging
    txt = normStep4(txt)
    # step5 : tagging for GANTU symbol
    txt = normStep5(txt)
    return txt

In [10]:
txt_norm_list = []
for file in file_list:
    txt = get_transcript(file)
    txt_norm = normalize(txt)
    
    txt_norm_list.append(txt_norm)

ERROR: 'euc_kr' codec can't decode byte 0x98 in position 68: illegal multibyte sequence


In [11]:
for i, file in enumerate(file_list[:10]):
    txt = get_transcript(file)
    txt_norm = normalize(txt)
    
    print("[{0:2d}] input : {1}".format(i, txt))
    print("     norm  : {}".format(txt_norm))
    print()

[ 0] input : 아/ 몬 소리야, 그건 또. b/
     norm  : 아[GANTU] 몬 소리야 그건 또 [NOISE:b]

[ 1] input : 나는 악습은 원래 없어진다+ 없어져야 된다고 생각하긴 했는데 근데 그/ 약간 필요악으로 하나 정도쯤은 있어야 되거든. 물 뜨러 가고.
     norm  : 나는 악습은 원래 없어진다[AMBIG:DUP] 없어져야 된다고 생각하긴 했는데 근데 그[GANTU] 약간 필요악으로 하나 정도쯤은 있어야 되거든 물 뜨러 가고

[ 2] input : b/ n/ 그래서 지호랑 계단 n/ 올라와서 b/ 막 위에 운동하는 기구 있대요. b/ 그서 그걸로 운동 할려구요. b/ n/
     norm  : [NOISE:b] [NOISE:n] 그래서 지호랑 계단 [NOISE:n] 올라와서 [NOISE:b] 막 위에 운동하는 기구 있대요 [NOISE:b] 그서 그걸로 운동 할려구요 [NOISE:b] [NOISE:n]

[ 3] input : 뭐/ 정신과 병원도 그 약 타서 먹어보고, 그 한동안 연락이 안 된 적이 있었단 말이야. 그때가 언제였 언제였더라?
     norm  : 뭐[GANTU] 정신과 병원도 그 약 타서 먹어보고 그 한동안 연락이 안 된 적이 있었단 말이야 그때가 언제였 언제였더라

[ 4] input : o/ b/ 그게 (0.1프로)/(영 점 일 프로) 가정의 아이들과 가정의 모습이야? b/
     norm  : [NOISE:o] [NOISE:b] 그게 영 점 일 프로 가정의 아이들과 가정의 모습이야 [NOISE:b]

[ 5] input : 그/ 친애하는 판사님께라는 법+ 법 관련 드라마 알고 있어?
     norm  : 그[GANTU] 친애하는 판사님께라는 법[AMBIG:DUP] 법 관련 드라마 알고 있어

[ 6] input : o/ 그래가지고 진짜 차 사야겠다 아니 뭐/ 차 안 되면 스쿠터라도 타야되겠다 막/ 그런 생각 들더라구 그래서 운전은 하는 게 좋은 거 같애 진짜 b/
     norm  :

### 2.2. Convert normalized transcript to lexical pronunciation symbol using G2P.

In [12]:
import sys
sys.path.insert(0, '..')   # To import parent dir's packages

from g2p import g2p
ver_info = sys.version_info

#### define Hangul / Symbols / Meta tags : for classification

In [13]:
HANGULS = ['ㅂ', 'ㅍ', 'ㅃ', 'ㄷ', 'ㅌ', 'ㄸ', 'ㄱ', 'ㅋ', 'ㄲ', 'ㅅ', 'ㅆ', 'ㅎ', 'ㅈ', 'ㅊ', 'ㅉ', 'ㅁ', 'ㄴ', 'ㄹ', 'ㅂ', 'ㅍ', 'ㄷ', 'ㅌ', 'ㄱ', 'ㅋ', 'ㄲ', 'ㅅ', 'ㅆ', 'ㅎ', 'ㅈ', 'ㅊ', 'ㅁ', 'ㄴ', 'ㅇ', 'ㄹ', 'ㄱㅅ', 'ㄴㅈ', 'ㄴㅎ', 'ㄹㄱ', 'ㄹㅁ', 'ㄹㅂ', 'ㄹㅅ', 'ㄹㅌ', 'ㄹㅍ', 'ㄹㅎ', 'ㅂㅅ', 'ㅣ', 'ㅔ', 'ㅐ', 'ㅏ', 'ㅡ', 'ㅓ', 'ㅜ', 'ㅗ', 'ㅖ', 'ㅒ', 'ㅑ', 'ㅕ', 'ㅠ', 'ㅛ', 'ㅟ', 'ㅚ', 'ㅙ', 'ㅞ', 'ㅘ', 'ㅝ', 'ㅢ']
SYMBOLS = ['p0', 'ph', 'pp', 't0', 'th', 'tt', 'k0', 'kh', 'kk', 's0', 'ss', 'h0', 'c0', 'ch', 'cc', 'mm', 'nn', 'rr', 'pf', 'ph', 'tf', 'th', 'kf', 'kh', 'kk', 's0', 'ss', 'h0', 'c0', 'ch', 'mf', 'nf', 'ng', 'll', 'ks', 'nc', 'nh', 'lk', 'lm', 'lb', 'ls', 'lt', 'lp', 'lh', 'ps', 'ii', 'ee', 'qq', 'aa', 'xx', 'vv', 'uu', 'oo', 'ye', 'yq', 'ya', 'yv', 'yu', 'yo', 'wi', 'wo', 'wq', 'we', 'wa', 'wv', 'xi']
META_TAGS = ['[AMBIG:DUP]', '[AMBIG:UNK]', '[GANTU]', '[NOISE:b]', '[NOISE:n]', '[NOISE:l]', '[NOISE:o]', '[SPACE]', '[UNK]']

* Symbol <-> Label indexing을 위한 Mapping Dataframe

In [14]:
df_korSym = pd.DataFrame()

df_korSym['Hangul'] = HANGULS
df_korSym['Symbol'] = SYMBOLS
for tag in META_TAGS:
    df_korSym.loc[len(df_korSym)] = [tag, tag]
df_korSym.head()

,Hangul,Symbol
0,ㅂ,p0
1,ㅍ,ph
2,ㅃ,pp
3,ㄷ,t0
4,ㅌ,th


#### define functions for prepare korean to pronunciation labels

In [15]:
def kor2pron(txt, rule_in, rule_out, special_symbol='|'):
    pron_list = []
    for w in txt.split():
        tag = re.findall('(\[.+?\])', w)
        if tag:
            w_base = w.replace(tag[0], '')
            prons = g2p.graph2prono(w_base, rule_in, rule_out)
            pron_list.append(special_symbol+prons+' '+tag[0])
        else:
            prons = g2p.graph2prono(w, rule_in, rule_out)
            pron_list.append(special_symbol+prons)
    pronun_txt = " ".join(pron_list)
    
    return pronun_txt

def pron_to_label(pron_txt, df_korSym, unk_flag=True):
    pron_txt = pron_txt[1:] if pron_txt.startswith('|') else pron_txt
    pron_txt = pron_txt.replace('|', '[SPACE] ')
    
    idx_list = []
    for tag in pron_txt.split():
        idx = df_korSym.index[df_korSym['Symbol'] == tag].tolist()[0]
        idx_list.append(idx)
    
    unk_idx = df_korSym.index[df_korSym['Symbol'] == '[UNK]'].tolist()[0]
    if unk_flag:
        idx_list = [unk_idx] + idx_list + [unk_idx]
    return idx_list

def label_to_pron(idx_list, df_korSym, with_space=True, unk_flag=True):
    pron_txt_list = []
    for idx in idx_list:
        pron = df_korSym.loc[idx]['Symbol']
        pron_txt_list.append(pron)
    if unk_flag:
        pron_txt_list = pron_txt_list[1:-1]
    
    pron_txt = " ".join(pron_txt_list)
    pron_txt = pron_txt.replace('[SPACE] ', '|')
    pron_txt = '|' + pron_txt
    return pron_txt

#### Generate pronunciation with g2p

In [16]:
# Load rulebook using g2p
[rule_in, rule_out] = g2p.readRules(ver_info[0], '../g2p/rulebook.txt')

In [17]:
pron_txt_list = [kor2pron(txt, rule_in, rule_out) for txt in txt_norm_list]

for txt in txt_norm_list[:3]:
    pron_txt = kor2pron(txt, rule_in, rule_out)
    labels = pron_to_label(pron_txt, df_korSym, True)
    print("="*100)
    print("> Origin Txt  : {}".format(txt))
    print("  Pronon Txt  : {}".format(pron_txt))
    print("  Labels      : {}".format(labels))
    print()

> Origin Txt  : 아[GANTU] 몬 소리야 그건 또 [NOISE:b]
  Pronon Txt  : |aa [GANTU] |mm oo nf |s0 oo rr ii ya |k0 xx k0 vv nf |tt oo | [NOISE:b]
  Labels      : [74, 48, 68, 73, 15, 52, 31, 73, 9, 52, 17, 45, 55, 73, 6, 49, 6, 50, 31, 73, 5, 52, 73, 69, 74]

> Origin Txt  : 나는 악습은 원래 없어진다[AMBIG:DUP] 없어져야 된다고 생각하긴 했는데 근데 그[GANTU] 약간 필요악으로 하나 정도쯤은 있어야 되거든 물 뜨러 가고
  Pronon Txt  : |nn aa nn xx nf |aa kf ss xx p0 xx nf |wv ll rr qq |vv pf ss vv c0 ii nf t0 aa [AMBIG:DUP] |vv pf ss vv c0 yv ya |t0 wo nf t0 aa k0 oo |s0 qq ng k0 aa kh aa k0 ii nf |h0 qq nf nn xx nf t0 ee |k0 xx nf t0 ee |k0 xx [GANTU] |ya kf kk aa nf |ph ii ll rr yo aa k0 xx rr oo |h0 aa nn aa |c0 vv ng t0 oo cc xx mm xx nf |ii ss vv ya |t0 wo k0 vv t0 xx nf |mm uu ll |tt xx rr vv |k0 aa k0 oo
  Labels      : [74, 16, 48, 16, 49, 31, 73, 48, 22, 10, 49, 0, 49, 31, 73, 64, 33, 17, 47, 73, 50, 18, 10, 50, 12, 45, 31, 3, 48, 66, 73, 50, 18, 10, 50, 12, 56, 55, 73, 3, 60, 31, 3, 48, 6, 52, 73, 9, 47, 32, 6, 48, 7, 48, 6, 45, 31, 73, 11, 47

In [18]:
# pron_txt = pron_txt_list[0]
# txt_norm = txt_norm_list[0]

# print(" Normalized text : ", txt_norm)
# print(" Pronunciation   : ", pron_txt)
# print()

# word_prons = pron_txt.split('|')
# decode_ch_list = []
# for word_pron in word_prons:
#     decode_ch_list.append('|')
#     for symbol in word_pron.split():
#         decode_ch_list.append(df[df["Symbol"] == symbol]["Hangul"].iloc[0])

# decoded_txt = "".join(decode_ch_list)
# decoded_txt = decoded_txt.replace("|", " ")
# print(decode_ch_list)
# print(decoded_txt)

## 3. Audio Preprocessing